In [26]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def update_neuroscope_colors(xml_input, xml_output):
    # Load XML file
    tree = ET.parse(xml_input)
    root = tree.getroot()

    # 1. Create a mapping between channels and groups
    channel_to_group = {}
    groups = root.findall('.//anatomicalDescription/channelGroups/group')
    num_groups = len(groups)
    
    if num_groups == 0:
        print("No groups found.")
        return

    for group_idx, group in enumerate(groups):
        for channel in group.findall('channel'):
            channel_id = channel.text.strip()
            channel_to_group[channel_id] = group_idx

    # 2. Generate color list from plasma colormap (Hex format)
    # Recommended method for matplotlib 3.5+
    cmap = plt.get_cmap('spring')
    group_hex_colors = [mcolors.to_hex(cmap(i / (num_groups - 1))) for i in range(num_groups)] if num_groups > 1 else [mcolors.to_hex(cmap(0.5))]

    # 3. Update color information in neuroscope section
    channel_colors_list = root.findall('.//neuroscope/channels/channelColors')
    
    for cc_element in channel_colors_list:
        chan_id_elem = cc_element.find('channel')
        if chan_id_elem is not None:
            chan_id = chan_id_elem.text.strip()
            
            # Apply color if the channel belongs to a group
            if chan_id in channel_to_group:
                g_idx = channel_to_group[chan_id]
                new_color = group_hex_colors[g_idx]
                
                # Update all related color tags
                for tag in ['color', 'anatomyColor', 'spikeColor']:
                    color_elem = cc_element.find(tag)
                    if color_elem is not None:
                        color_elem.text = new_color

    # 4. Format and save XML (indent is available in Python 3.9+)
    if hasattr(ET, 'indent'):
        ET.indent(tree, space="  ", level=0)
    
    tree.write(xml_output, encoding='utf-8', xml_declaration=True)
    print(f"Processing completed. Output saved to: {xml_output}")
    print(f"Assigned group colors (plasma): {group_hex_colors}")


In [15]:
import os, sys
sys.path.append(os.path.abspath(".."))
from analysis.io import select_folder
xml_dir = select_folder(initial_drive="S:\\")
xml_path = os.path.join(xml_dir, "amplifier.xml")

Selected folder: S:/data/theta_replay/Sake/sake_testday02/sake_test_recording_251226_113940


In [27]:
update_neuroscope_colors(xml_path, xml_path)

Processing completed. Output saved to: S:/data/theta_replay/Sake/sake_testday02/sake_test_recording_251226_113940\amplifier.xml
Assigned group colors (plasma): ['#ff00ff', '#ff24db', '#ff49b6', '#ff6d92', '#ff926d', '#ffb649', '#ffdb24', '#ffff00']
